In [1]:
%matplotlib inline
import matplotlib
import seaborn as sns
matplotlib.rcParams['savefig.dpi'] = 144

In [2]:
from static_grader import grader

# PW Miniproject
## Introduction

The objective of this miniproject is to exercise your ability to use basic Python data structures, define functions, and control program flow. We will be using these concepts to perform some fundamental data wrangling tasks such as joining data sets together, splitting data into groups, and aggregating data into summary statistics.
**Please do not use `pandas` or `numpy` to answer these questions.**

We will be working with medical data from the British NHS on prescription drugs. Since this is real data, it contains many ambiguities that we will need to confront in our analysis. This is commonplace in data science, and is one of the lessons you will learn in this miniproject.

## Downloading the data

We first need to download the data we'll be using from Amazon S3:

In [3]:
%%bash
mkdir pw-data
wget http://dataincubator-wqu.s3.amazonaws.com/pwdata/201701scripts_sample.json.gz -nc -P ./pw-data
wget http://dataincubator-wqu.s3.amazonaws.com/pwdata/practices.json.gz -nc -P ./pw-data

mkdir: cannot create directory ‘pw-data’: File exists
File ‘./pw-data/201701scripts_sample.json.gz’ already there; not retrieving.

File ‘./pw-data/practices.json.gz’ already there; not retrieving.



## Loading the data

The first step of the project is to read in the data. We will discuss reading and writing various kinds of files later in the course, but the code below should get you started.

In [3]:
import gzip
import simplejson as json

In [4]:
with gzip.open('./pw-data/201701scripts_sample.json.gz', 'rb') as f:
    scripts = json.load(f)

with gzip.open('./pw-data/practices.json.gz', 'rb') as f:
    practices = json.load(f)

This data set comes from Britain's National Health Service. The `scripts` variable is a list of prescriptions issued by NHS doctors. Each prescription is represented by a dictionary with various data fields: `'practice'`, `'bnf_code'`, `'bnf_name'`, `'quantity'`, `'items'`, `'nic'`, and `'act_cost'`. 

In [5]:
scripts[:2]

[{'bnf_code': '0101010G0AAABAB',
  'items': 2,
  'practice': 'N81013',
  'bnf_name': 'Co-Magaldrox_Susp 195mg/220mg/5ml S/F',
  'nic': 5.98,
  'act_cost': 5.56,
  'quantity': 1000},
 {'bnf_code': '0101021B0AAAHAH',
  'items': 1,
  'practice': 'N81013',
  'bnf_name': 'Alginate_Raft-Forming Oral Susp S/F',
  'nic': 1.95,
  'act_cost': 1.82,
  'quantity': 500}]

A [glossary of terms](http://webarchive.nationalarchives.gov.uk/20180328130852tf_/http://content.digital.nhs.uk/media/10686/Download-glossary-of-terms-for-GP-prescribing---presentation-level/pdf/PLP_Presentation_Level_Glossary_April_2015.pdf/) and [FAQ](http://webarchive.nationalarchives.gov.uk/20180328130852tf_/http://content.digital.nhs.uk/media/10048/FAQs-Practice-Level-Prescribingpdf/pdf/PLP_FAQs_April_2015.pdf/) is available from the NHS regarding the data. Below we supply a data dictionary briefly describing what these fields mean.

| Data field |Description|
|:----------:|-----------|
|`'practice'`|Code designating the medical practice issuing the prescription|
|`'bnf_code'`|British National Formulary drug code|
|`'bnf_name'`|British National Formulary drug name|
|`'quantity'`|Number of capsules/quantity of liquid/grams of powder prescribed|
| `'items'`  |Number of refills (e.g. if `'quantity'` is 30 capsules, 3 `'items'` means 3 bottles of 30 capsules)|
|  `'nic'`   |Net ingredient cost|
|`'act_cost'`|Total cost including containers, fees, and discounts|

The `practices` variable is a list of member medical practices of the NHS. Each practice is represented by a dictionary containing identifying information for the medical practice. Most of the data fields are self-explanatory. Notice the values in the `'code'` field of `practices` match the values in the `'practice'` field of `scripts`.

In [15]:
practices[:2]

[{'code': 'A81001',
  'name': 'THE DENSHAM SURGERY',
  'addr_1': 'THE HEALTH CENTRE',
  'addr_2': 'LAWSON STREET',
  'borough': 'STOCKTON ON TEES',
  'village': 'CLEVELAND',
  'post_code': 'TS18 1HU'},
 {'code': 'A81002',
  'name': 'QUEENS PARK MEDICAL CENTRE',
  'addr_1': 'QUEENS PARK MEDICAL CTR',
  'addr_2': 'FARRER STREET',
  'borough': 'STOCKTON ON TEES',
  'village': 'CLEVELAND',
  'post_code': 'TS18 2AW'}]

In the following questions we will ask you to explore this data set. You may need to combine pieces of the data set together in order to answer some questions. Not every element of the data set will be used in answering the questions.

## Question 1: summary_statistics

Our beneficiary data (`scripts`) contains quantitative data on the number of items dispensed (`'items'`), the total quantity of item dispensed (`'quantity'`), the net cost of the ingredients (`'nic'`), and the actual cost to the patient (`'act_cost'`). Whenever working with a new data set, it can be useful to calculate summary statistics to develop a feeling for the volume and character of the data. This makes it easier to spot trends and significant features during further stages of analysis.

Calculate the sum, mean, standard deviation, and quartile statistics for each of these quantities. Format your results for each quantity as a list: `[sum, mean, standard deviation, 1st quartile, median, 3rd quartile]`. We'll create a `tuple` with these lists for each quantity as a final result.

In [7]:
"""Statistics Method"""
import math
def mean(data):
    n = len(data)
    return sum(data)/n

def _ss(data):
    """Return sum of square deviations of sequence data."""
    c = mean(data)
    ss = sum((x-c)**2 for x in data)
    return ss

def sd(data):
    return math.sqrt(_ss(data)/(len(data)-1))

def mode(data):
    return max(set(data), key = data.count)

def median(data):
    sort = sorted(data)
    l = len(sort)
    mid = (l - 1) // 2
    if (l % 2):
        return sort[mid]
    else:
        return (sort[mid] + sort[mid + 1])/2

def quartile(data):
    sort = sorted(data)
    l = len(sort)
    mid = (l + 1) // 2
    if mid == (l + 1) // 2:
#        print(sort[:mid],sort[mid:])
        return median(sort[:mid]),median(sort[mid:])
    else :
#        print(sort[:mid],sort[mid+1:])
        return median(sort[:mid]),median(sort[mid + 1:])


In [10]:
def get_keys(lst):
    return [key for key in lst[0].keys()] 
def make_data_dic(lst):
    data_dic = {}
    data_keys = get_keys(lst)
    
    """Make EMPTY LIST of Data Dictionary of each Keys"""
    for key in data_keys:
        data_dic[key] = []
    """Add All the Data to Specific key by opening each entry in First Loop & add on data_dic in next loop"""
    for entry in lst:
        for key in data_keys:
            data_dic[key].append(entry[key]) 
    return data_dic

scripts_list = make_data_dic(scripts)

In [14]:
def describe(key):
    key_data = scripts_list[key]
    total = sum(key_data)
    m = mean(key_data)
    md = median(key_data)
    sds = sd(key_data)
    first, third = quartile(key_data)
    return total, m ,sds,first,md,third



In [15]:
summary = [('items', describe('items')),
           ('quantity', describe('quantity')),
           ('nic', describe('nic')),
           ('act_cost', describe('act_cost'))]

In [16]:
grader.score.pw__summary_statistics(summary)

Your score:  1.0


## Question 2: most_common_item

Often we are not interested only in how the data is distributed in our entire data set, but within particular groups -- for example, how many items of each drug (i.e. `'bnf_name'`) were prescribed? Calculate the total items prescribed for each `'bnf_name'`. What is the most commonly prescribed `'bnf_name'` in our data?

To calculate this, we first need to split our data set into groups corresponding with the different values of `'bnf_name'`. Then we can sum the number of items dispensed within in each group. Finally we can find the largest sum.

We'll use `'bnf_name'` to construct our groups. You should have *5619* unique values for `'bnf_name'`.

In [13]:
def _unique_entry(my_list):
    return sorted(list(set(my_list)))


In [21]:
bnf_names = _unique_entry(scripts_list['bnf_name'])
assert(len(bnf_names) == 5619)

We want to construct "groups" identified by `'bnf_name'`, where each group is a collection of prescriptions (i.e. dictionaries from `scripts`). We'll construct a dictionary called `groups`, using `bnf_names` as the keys. We'll represent a group with a `list`, since we can easily append new members to the group. To split our `scripts` into groups by `'bnf_name'`, we should iterate over `scripts`, appending prescription dictionaries to each group as we encounter them.

In [26]:
groups = {name: [] for name in bnf_names}
for script in scripts:
    # INSERT ...
    groups[script['bnf_name']].append(script['items'])

In [31]:
sum_groups = {g : sum(groups[g]) for g in groups}

In [24]:
def dictionay_to_truple(dictionay):
    "Input[dic]: {a:1,b:2,c:2}, Output[list of tuple]: (a,1),(b,2),(c,3)"
    return [(k,v) for k,v in dictionay.items()]

In [35]:
sum_groups = dictionay_to_truple(sum_groups)

Now that we've constructed our groups we should sum up `'items'` in each group and find the `'bnf_name'` with the largest sum. The result, `max_item`, should have the form `[(bnf_name, item total)]`, e.g. `[('Foobar', 2000)]`.

In [42]:
max_item = 
max_item

[('Omeprazole_Cap E/C 20mg', 113826)]

**TIP:** If you are getting an error from the grader below, please make sure your answer conforms to the correct format of `[(bnf_name, item total)]`.

In [43]:
grader.score.pw__most_common_item(max_item)

Your score:  1.0


**Challenge:** Write a function that constructs groups as we did above. The function should accept a list of dictionaries (e.g. `scripts` or `practices`) and a tuple of fields to `groupby` (e.g. `('bnf_name')` or `('bnf_name', 'post_code')`) and returns a dictionary of groups. The following questions will require you to aggregate data in groups, so this could be a useful function for the rest of the miniproject.

In [20]:
from itertools import combinations 
def rSubset(arr, r):
    return list(combinations(arr, r))
if __name__ == "__main__": 
    arr = [1, 2, 3, 4] 
    r = 2
    print (rSubset(arr, r) )


[(1, 2), (1, 3), (1, 4), (2, 3), (2, 4), (3, 4)]


In [18]:
import itertools as it

my_dict=uniqe_dictionay(scripts[:20], ('bnf_name','practice'))
def _combinations(dictionary):
    allNames = dictionary
    return list(set(it.product(*(dictionary[Name] for Name in allNames))))

print(_combinations(my_dict))

[('Alginate_Raft-Forming Oral Susp S/F', 'N81013'), ('Colpermin_Cap E/C 0.2ml M/R', 'N81013'), ('Mebeverine HCl_Tab 135mg', 'N81013'), ('Ispag/Mebeverine_Gran Eff 3.5g/135mg S/F', 'N81013'), ('Gaviscon Infant_Sach 2g (Dual Pack) S/F', 'N81013'), ('Gaviscon Advance_Tab Chble Mint(Reckitt)', 'N81013'), ('Ranitidine HCl_Tab 150mg', 'N81013'), ('Hyoscine Butylbrom_Inj 20mg/ml 1ml Amp', 'N81013'), ('Co-Magaldrox_Susp 195mg/220mg/5ml S/F', 'N81013'), ('Peptac_Liq (Peppermint) S/F', 'N81013'), ('Gaviscon Advance_Liq (Aniseed) (Reckitt)', 'N81013'), ('Peppermint Oil_Cap E/C 0.2ml', 'N81013'), ('Gaviscon Advance_Liq (Peppermint) S/F', 'N81013'), ('Peppermint Oil_Cap E/C 0.2ml M/R', 'N81013'), ('Fybogel Mebeverine_Eff Gran Sach S/F', 'N81013'), ('Alverine Cit_Cap 60mg', 'N81013'), ('Hyoscine Butylbrom_Tab 10mg', 'N81013'), ('Sod Alginate/Pot Bicarb_Tab Chble 500mg', 'N81013'), ('Sod Algin/Pot Bicarb_Susp S/F', 'N81013'), ('Mebeverine HCl_Cap 200mg M/R', 'N81013')]


In [6]:
import itertools as it

def _combinations(dictionary):
    allNames = dictionary
    return list(set(it.product(*(dictionary[Name] for Name in allNames))))

def get_keys(lst):
    return [key for key in lst[0].keys()] 

def make_data_dic(lst):
    data_dic = {}
    data_keys = get_keys(lst)
    
    """Make EMPTY LIST of Data Dictionary of each Keys"""
    for key in data_keys:
        data_dic[key] = []
    """Add All the Data to Specific key by opening each entry in First Loop & add on data_dic in next loop"""
    for entry in lst:
        for key in data_keys:
            data_dic[key].append(entry[key]) 
    return data_dic

def remove_empty_keys(g):
    return {k:v for k,v in g.items() if len(v) != 0 }

def _unique_entry(my_list):
    return sorted(list(set(my_list)))

def uniqe_dictionay(data, fields):
    dictionay = {f:_unique_entry(make_data_dic(data)[f]) for f in fields}
    return dictionay

def group_by_field(data, fields,group_by = 'items'):
    fields = list(fields)
    unq_dictionay = uniqe_dictionay(data, fields)
    groups = {key: [] for key in _combinations(unq_dictionay)}
    for d in data:
        l = tuple([d[key] for key in list(fields)])
        groups[l].append(d[group_by])
    return remove_empty_keys(groups)

In [16]:
g = group_by_field(scripts, ('bnf_name',))

In [43]:
def tuple_to_dictionay(lst):
    return {k:v for k,v in lst}

def dictionay_to_truple(dictionay):
    "Input[dic]: {a:1,b:2,c:2}, Output[list of tuple]: (a,1),(b,2),(c,3)"
    return [(k,v) for k,v in dictionay.items()]

def tuple_max(tuples):
    "Input[tuple list]: [(a:1),(b:2),(c:3)], Output[max tuple]: [(c,3)]"
    return max(tuples,key = lambda index:index[1])

def tuple_sorted(tuples,by = 1,reverse=False):
    "Input[tuple list]: [(a:1),(b:2),(c:3)], Output[max tuple]: [(c,3),(b:2),(a:1)]"
    if reverse == False:
        return sorted(tuples,key = lambda index:index[by],reverse = False)
    else:
        return sorted(tuples,key = lambda index:index[by],reverse = True)

def max_items(dictionay):
    "Input[dic]: {a:[1,1,1],b:[2,2,2]}, Output[max sum of list of tuple]: [(b,3)]"
    dictionay = {key : sum(dictionay[key]) for key in dictionay}
    return tuple_max(dictionay_to_truple(dictionay))

def clean_dictionay_sorted_first(dictionay):
    "Input[dic]: {(a,):[4,5,2],(b,):[1,2,3]}, Output[dic]: a:2,b:1"
    return {key[0]: sorted(value)[0] for key,value in dictionay.items()}

def clean_dictionay_sum(dictionay,k = 0):
    "Input[dic]: {(a,):[4,5,2],(b,):[1,2,3]}, Output[dic]: a:11,b:6"
    if k == 0:
        return {key[k]: sum(value) for key,value in dictionay.items()}
    else:
        return {key: sum(value) for key,value in dictionay.items()}

In [18]:
g = max_items(g)
g

[(('Omeprazole_Cap E/C 20mg',), 113826)]

In [62]:
sample = scripts

In [68]:
groups = group_by_field(scripts, 'bnf_name')
test_max_item = max_items(groups)
assert test_max_item == max_item
test_max_item

[('Omeprazole_Cap E/C 20mg', 113826)]

## Question 3: postal_totals

Our data set is broken up among different files. This is typical for tabular data to reduce redundancy. Each table typically contains data about a particular type of event, processes, or physical object. Data on prescriptions and medical practices are in separate files in our case. If we want to find the total items prescribed in each postal code, we will have to _join_ our prescription data (`scripts`) to our clinic data (`practices`).

Find the total items prescribed in each postal code, representing the results as a list of tuples `(post code, total items prescribed)`. Sort your results ascending alphabetically by post code and take only results from the first 100 post codes. Only include post codes if there is at least one prescription from a practice in that post code.

**NOTE:** Some practices have multiple postal codes associated with them. Use the alphabetically first postal code.

We can join `scripts` and `practices` based on the fact that `'practice'` in `scripts` matches `'code'` in `practices`. However, we must first deal with the repeated values of `'code'` in `practices`. We want the alphabetically first postal codes.

In [8]:
practice_postal = clean_dictionay_sorted_first(group_by_field(practices, ('code',),'post_code'))

In [ ]:
practice_postal = {}
for practice in practices:
    if practice['code'] in practice_postal:
        practice_postal[practice['code']] = ...
    else:
        practice_postal[practice['code']] = ...

**Challenge:** This is an aggregation of the practice data grouped by practice codes. Write an alternative implementation of the above cell using the `group_by_field` function you defined previously.

In [9]:
assert practice_postal['K82019'] == 'HP21 8TR'

**Challenge:** This is an aggregation of the practice data grouped by practice codes. Write an alternative implementation of the above cell using the `group_by_field` function you defined previously.

In [10]:
assert practice_postal['K82019'] == 'HP21 8TR'

Now we can join `practice_postal` to `scripts`.

In [11]:
joined = scripts[:]
for script in joined:
    script['post_code'] = practice_postal[script['practice']]

Finally we'll group the prescription dictionaries in `joined` by `'post_code'` and sum up the items prescribed in each group, as we did in the previous question.

In [50]:
items_by_post = group_by_field(joined, ('post_code',))
items_by_post = clean_dictionay_sum(items_by_post)
print(items_by_post['B11 4BW'])
items_by_post = dictionay_to_truple(items_by_post)
len(items_by_post)

20673


118

In [53]:
tuple_sorted(items_by_post,0)[:10]

[('B11 4BW', 20673),
 ('B18 7AL', 19001),
 ('B21 9RY', 29103),
 ('B23 6DJ', 24859),
 ('B70 7AW', 36531),
 ('BB11 2DL', 34356),
 ('BB2 1AX', 28254),
 ('BB3 1PY', 54514),
 ('BB4 5SL', 29388),
 ('BB7 2JG', 44585)]

In [57]:
postal_totals = tuple_sorted(items_by_post,0)[:100]
len(postal_totals)

100

In [59]:
grader.score.pw__postal_totals(postal_totals)

Your score:  1.0


## Question 4: items_by_region

Now we'll combine the techniques we've developed to answer a more complex question. Find the most commonly dispensed item in each postal code, representing the results as a list of tuples (`post_code`, `bnf_name`, amount dispensed as proportion of total). Sort your results ascending alphabetically by post code and take only results from the first 100 post codes.

**NOTE:** We'll continue to use the `joined` variable we created before, where we've chosen the alphabetically first postal code for each practice. Additionally, some postal codes will have multiple `'bnf_name'` with the same number of items prescribed for the maximum. In this case, we'll take the alphabetically first `'bnf_name'`.

Now we need to calculate the total items of each `'bnf_name'` prescribed in each `'post_code'`. Use the techniques we developed in the previous questions to calculate these totals. You should have 141196 `('post_code', 'bnf_name')` groups.

In [13]:
groups = group_by_field(joined, ('post_code', 'bnf_name'))

In [14]:
total_items_by_bnf_post = groups
assert len(total_items_by_bnf_post) == 141196

Let's use `total_items` to find the maximum item total for each postal code. To do this, we will want to regroup `total_items_by_bnf_post` by `'post_code'` only, not by `('post_code', 'bnf_name')`. First let's turn `total_items` into a list of dictionaries (similar to `scripts` or `practices`) and then group it by `'post_code'`. You should have 118 groups in the resulting `total_items_by_post` after grouping `total_items` by `'post_code'`.

In [15]:
total_items_by_post = items_by_post
assert len(total_items_by_post) == 118

In [16]:
total_items_by_post_100 = tuple_sorted(total_items_by_post,0,False)[:100]
total_items_by_post_100 = tuple_to_dictionay(total_items_by_post_100)

Now we will aggregate the groups in `total_by_item_post` to create `max_item_by_post`. Some `'bnf_name'` have the same item total within a given postal code. Therefore, if more than one `'bnf_name'` has the maximum item total in a given postal code, we'll take the alphabetically first `'bnf_name'`. We can do this by [sorting](https://docs.python.org/2.7/howto/sorting.html) each group according to the item total and `'bnf_name'`.

In [22]:
sums = clean_dictionay_sum(total_items_by_bnf_post,1)

In [30]:
l = {name: [] for name in total_items_by_post_100.keys()}
for j in sums.keys():
    if j[0] in l.keys():
        l[j[0]].append(tuple([j[1],sums[j]]))

In [65]:
def max_dic(dictionay):
    r = []
    for e in dictionay:
        if e in total_items_by_post_100.keys():
            tp = tuple_max(dictionay[e])
            r.append(tuple([e,tp[0],tp[1]/total[e]])) 
    return r

In [62]:
total = tuple_to_dictionay(postal_totals)

In [69]:
items_by_region = max_dic(l)
items_by_region

[('B11 4BW', 'Salbutamol_Inha 100mcg (200 D) CFF', 0.03415082474725487),
 ('B18 7AL', 'Salbutamol_Inha 100mcg (200 D) CFF', 0.02926161780958897),
 ('B21 9RY', 'Metformin HCl_Tab 500mg', 0.03549462254750369),
 ('B23 6DJ', 'Lansoprazole_Cap 30mg (E/C Gran)', 0.024095900880968663),
 ('B70 7AW', 'Paracet_Tab 500mg', 0.0266896608360023),
 ('BB11 2DL', 'Omeprazole_Cap E/C 20mg', 0.02884503434625684),
 ('BB2 1AX', 'Omeprazole_Cap E/C 20mg', 0.03645501521908402),
 ('BB3 1PY', 'Omeprazole_Cap E/C 20mg', 0.03428477088454342),
 ('BB4 5SL', 'Omeprazole_Cap E/C 20mg', 0.040696883081529876),
 ('BB7 2JG', 'Omeprazole_Cap E/C 20mg', 0.029471795446899183),
 ('BB8 0JZ', 'Atorvastatin_Tab 20mg', 0.022563442442074293),
 ('BB9 7SR', 'Omeprazole_Cap E/C 20mg', 0.023833193804939305),
 ('BD3 8QH', 'Atorvastatin_Tab 40mg', 0.03422179914326511),
 ('BH18 8EE', 'Omeprazole_Cap E/C 20mg', 0.029000583563798747),
 ('BH23 3AF', 'Omeprazole_Cap E/C 20mg', 0.03733292364418497),
 ('BL1 8TU', 'Omeprazole_Cap E/C 20mg', 0

In [ ]:
max_item_by_post = ...

In order to express the item totals as a proportion of the total amount of items prescribed across all `'bnf_name'` in a postal code, we'll need to use the total items prescribed that we previously calculated as `items_by_post`. Calculate the proportions for the most common `'bnf_names'` for each postal code. Format your answer as a list of tuples: `[(post_code, bnf_name, total)]`

In [ ]:
items_by_region = [('B11 4BW', 'Salbutamol_Inha 100mcg (200 D) CFF', 0.0341508247)] * 100

In [70]:
grader.score.pw__items_by_region(items_by_region)

Your score:  1.0


*Copyright &copy; 2017 The Data Incubator.  All rights reserved.*